In [1]:
from pyGROMACS import GROMACS_setup
from pyGROMACS.utils_automated import read_out_optimized_lambdas

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



In [ ]:
## change read xvg again

# pyGROMACS

This notebook enables the setup of MD simulations using GROMACS and to analyse them in an automated fashion

In [11]:
gromacs_setup = GROMACS_setup( system_setup = "input/system_setup_C6.yaml", 
                               simulation_default = "/home/st/st_us-041610/st_ac137577/software/pyGROMACS/parameters/parameter_defaults.yaml",
                               simulation_ensemble = "/home/st/st_us-041610/st_ac137577/software/pyGROMACS/parameters/parameter_ensemble.yaml",
                               submission_command = "sbatch"
                             )

## Setting up a simulation pipeline

In this section the possibility to setup a simulation folder, along with a simulation pipeline using several ensembles is set up.

1) Setup simulation and build initial system (if not provided)
2) Submit jobs to cluster

In [12]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "nvt", "npt_equilibration", "npt_production", "nvt" ]

# Define the simulation time per ensemble in nano seconds (for em provide number of iterations)
simulation_times = [ 10000, 1.0, 20.0, 10.0, 20.0 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# In that case, provide one initial structure for each temperature & pressure state.
# If the simulation should start from an initial configuration, provide an empty list.
initial_systems = [  ] 
initial_systems = []

# Provide kwargs that should be passed into the mdp template directly
mdp_kwargs = { "nemd": { "cos_acceleration": 0.2 } }
mdp_kwargs = {}

# Define number of copies
copies = 2

# Define if the inital system should build locally or with the cluster
on_cluster = False


gromacs_setup.prepare_simulation( folder_name = simulation_folder, ensembles = ensembles, simulation_times = simulation_times,
                                  initial_systems = initial_systems, mdp_kwargs = mdp_kwargs, copies = copies,
                                  on_cluster = on_cluster )


Building system based on provided molecule numbers and coordinate files!


Build system locally! Wait until it is finished.

Build successful



In [10]:
# Submit the simulations
gromacs_setup.submit_simulation()


Submitting simulations at Temperature = 313 K, Pressure = 1 bar

Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_313_pres_1/copy_0/job_313.sh
Submitted batch job 23254580


Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_313_pres_1/copy_1/job_313.sh
Submitted batch job 23254581


Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_313_pres_1/copy_2/job_313.sh
Submitted batch job 23254582



Submitting simulations at Temperature = 323 K, Pressure = 1 bar

Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_323_pres_1/copy_0/job_323.sh
Submitted batch job 23254583


Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_323_pres_1/copy_1/job_323.sh
Submitted batch job 23254584


Submitting job: /home/st/st_us-041610/st_ac137577/fair_simulations/C5/md_thermo/temp_323_pres_1/copy_2/job_323.sh
Submitted batch job 23254585


## Scanning (solvation) free energy

This part covers the scanning of solvation free energy.

1) Add Guest molecule in equilibrated system and equilibrate it further. !! Solutes should have the same name as in the topology file !!
2) [Optional] Optimize the intermediates for decoupling
3) Submit decoupling simulations for each solute

In [ ]:
## get again that job files are corretcly submitted

### 1) Add guest molecules and equilibrate the system

In [ ]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "nvt", "npt_equilibration", "npt_production" ]

# Define the simulation time per ensemble in nano seconds (for em provide number of iterations)
simulation_times = [ 10000, 2.0, 2.0, 10.0 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# In that case, provide one initial structure for each temperature & pressure state.
# If the simulation should start from an initial configuration, provide an empty list.
initial_systems = [  ] 
initial_systems = []

# Define solutes (same naming as in the topology file!) and their corresponding coordinates
solutes = [ "CO2", "N2", "CH4", "O2" ]
solute_coordinates = [ "/home/st/st_us-041610/st_ac137577/fair_simulations/coordinates/co2.gro",
                       "/home/st/st_us-041610/st_ac137577/fair_simulations/coordinates/n2.gro",
                       "/home/st/st_us-041610/st_ac137577/fair_simulations/coordinates/met.gro",
                       "/home/st/st_us-041610/st_ac137577/fair_simulations/coordinates/o2.gro" ]

# Define number of copies
copies = 0

# Ensure that all jobs for all solutes are submitted at once
job_files = [ [] for _ in gromacs_setup.system_setup["system"]["temperature"] ]

for solute, solute_coordinate in zip( solutes, solute_coordinates ):
    gromacs_setup.add_guest_molecule_and_prepare_equilibrate( folder_name = simulation_folder, ensembles = ensembles, simulation_times = simulation_times,
                                                              initial_systems = initial_systems, solute = solute, solute_coordinate = solute_coordinate, 
                                                              copies = copies )
    for i, files in enumerate( gromacs_setup.job_files ):
        job_files[i].extend( files )

gromacs_setup.job_files = job_files

In [ ]:
# Submit the simulations
gromacs_setup.submit_simulation()

### 2) [Optional] Optimization of intermediates


In [ ]:
# Define simulation folder
simulation_folder = "free_energy"

# Solutes for optimization
solutes           = [ "CO2", "O2", "CH4", "N2" ]

# Choose one state point for optimization (should not vary much over temperature and thus, one should be enough)
temperature       = gromacs_setup.system_setup["system"]["temperature"][0]
pressure          = gromacs_setup.system_setup["system"]["pressure"][0]
compressibility   = gromacs_setup.system_setup["system"]["compressibility"][0]

for solute in solutes:

    initial_cord  = f'{gromacs_setup.system_setup["system"]["folder"]}/{gromacs_setup.system_setup["system"]["name"]}/free_energy/{solute}/equilibration/temp_{temperature:.0f}_pres_{pressure:.0f}/copy_0/03_prod/prod.gro'
    initial_cpt   = f'{gromacs_setup.system_setup["system"]["folder"]}/{gromacs_setup.system_setup["system"]["name"]}/free_energy/{solute}/equilibration/temp_{temperature:.0f}_pres_{pressure:.0f}/copy_0/03_prod/prod.cpt'
    initial_topo  = f'{gromacs_setup.system_setup["system"]["folder"]}/{gromacs_setup.system_setup["system"]["name"]}/free_energy/{solute}/box/topology_{gromacs_setup.system_setup["system"]["name"]}.top'

    gromacs_setup.optimize_intermediates( folder_name = simulation_folder, simulation_free_energy = "parameter_free_energy.yaml", solute = solute,
                                             temperature = temperature, pressure = pressure, compressibility = compressibility, iteration_time = 1.0,
                                             initial_coord = initial_cord, initial_cpt = initial_cpt, initial_topo = initial_topo, tolerance = 0.10,
                                             min_overlap = 0.25, max_overlap = 0.4 )


In [ ]:
# Read out optimized lambdas
combined_lambdas = []

solutes           = [ "CO2", "O2", "CH4", "N2" ]
    
for i,solute in enumerate(solutes):
    log_opt_file = f'{gromacs_setup.system_setup["system"]["folder"]}/{gromacs_setup.system_setup["system"]["name"]}/free_energy/{solute}/optimization/opt_intermediates.log'
    combined_lambdas.append( read_out_optimized_lambdas(log_opt_file) )
    print(f"Optimized lambdas for {solute}:\n   {' '.join([str(l) for l in combined_lambdas[i]])}\n")

### 3) Submit final free energy jobs

In [ ]:
# Setup free energy simulations

# Define simulation folder
simulation_folder = "free_energy"

# Define ensembles, the meaning of each ensemble is defined in the simulation_ensemble yaml
ensembles        = [ "em", "nvt", "npt", "prod" ]

# Define simulation time per ensemble in nanoseconds (for "em" provide number of optimization steps)
simulation_times = [ 10000, 2.0, 2.0, 10.0 ]

# Define the number of copies for each system
copies           = 0

# Solutes
solutes          = [ "CO2", "O2", "CH4", "N2" ]

# Define lambdas that should be used (for each solute individual)
#combined_lambdas = [[ 0.069, 0.33 , 0.67 , 0.931, 1.025, 1.129, 1.297, 1.5  , 1.703, 1.871, 1.975 ]]

# Ensure that all jobs for all solutes are submitted at once
job_files = [ [] for _ in gromacs_setup.system_setup["system"]["temperature"] ]

for i,solute in enumerate( solutes ):

    # Define initial systems for each temperature & pressure state, based on the equilibration done before
    initial_systems = [ ( f'{gromacs_setup.system_setup["system"]["folder"]}/{gromacs_setup.system_setup["system"]["name"]}'
                          f'/free_energy/{solute}/equilibration/temp_{temp:.0f}_pres_{press:.0f}/copy_0/03_prod/prod.gro' )
                        for temp, press in zip( gromacs_setup.system_setup["system"]["temperature"], gromacs_setup.system_setup["system"]["pressure"] ) ]


    gromacs_setup.prepare_free_energy_simulation( folder_name = simulation_folder, simulation_free_energy = "parameter_free_energy.yaml", solute = solute, 
                                                      combined_lambdas = combined_lambdas[i], initial_systems = initial_systems, ensembles = ensembles,
                                                      copies = copies, simulation_times = simulation_times  )

    for j, files in enumerate( gromacs_setup.job_files ):
      job_files[j].extend( files )

gromacs_setup.job_files = job_files

In [ ]:
# Submit the simulations
gromacs_setup.submit_simulation()

## Data analysis

This generic tool allows to extract properties using GROMACS commands for an specified simulation folder and ensemble. This will average the properties over each copy and safe the produced data in each temperature & pressure folder as json file

### Extract and analysis generic properties

In [ ]:
# Extract properties from GROMACS and analyse them

# Define analysis folder
analysis_folder = "md_thermo"

# Define analysis ensemble
ensemble = "03_nvt" # "02_npt" # 

# Define GROMACS command
command = "msd" # "energy" #  

# Properties to extract
properties = ["SOL"] # ["Density", "Volume", "Enthalpy"] #  ["1/Viscosity"] # 

# Special args to gromacs analysis
args = [ "-dt 2", "-beginfit 5000", "-endfit 17500", "-type x"] # [ ] # 

# XVG output name
output_name = "msd_x" # "properties" # 

# Submit analysis on cluster
on_cluster = False

# Perform extraction from gromacs
extract = True

gromacs_setup.analysis_extract_properties( analysis_folder = analysis_folder, ensemble = ensemble, extracted_properties = properties, fraction = 0.25,
                                           command = command, args = args, output_name = output_name, on_cluster = on_cluster, extract = extract )

### Extract and analysis solvation free energies

In [ ]:
# Extract solvation free energy results 

# Define analysis folder
analysis_folder = "free_energy"

# Define analysis ensemble
ensemble = "03_prod"

# Solutes
solutes  = [ "CO2" ]

for solute in solutes:
    gromacs_setup.analysis_free_energy( analysis_folder = analysis_folder, ensemble = ensemble, solute = solute )